In [1]:
# Azure OpenAI Assistants API with functions

import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Load environment variables from .env file
# AZURE_OPENAI_API_KEY
# AZURE_OPENAI_ENDPOINT
# AZURE_OPENAI_API_VERSION
# SEARCH_KEY
load_dotenv()

# Create Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION')
)

# assistant ID as created in the portal
assistant_id = "aasst_NjjVJ5GVWXjJLvYvXtmEQsQR"

## Create a thread

A thread is not linked to the assistant at creation time.

In [2]:
# Create a thread
thread = client.beta.threads.create()

# Threads have an id as well
print("Thread id: ", thread.id)

Thread id:  thread_5sdhXFzj4pCruE4627Dp4qAy


## Add a message to the thread

The assistant can perform actions like turning a lamp on or off or setting its brightness.

The function json can be found in the assistant definition in the Assistant Playgrond.

- set_lamp(lamp_id, state)
- set_lamp_brightness(lamp_id, brightness)

After adding the user message and running the thread, we show information about the run via a json dump.

Important: do not wait too long with the other cells because you need to provide a tool response within a certain amount of time.

In [3]:
import time
from IPython.display import clear_output

# function returns the run when status is no longer queued or in_progress
def wait_for_run(run, thread_id):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
        )
        time.sleep(0.5)

    return run


# create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Turn living room lamp and kitchen lamp on. Set both lamps to half brightness."
)

# create a run 
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id # use the assistant id defined in the first cell
)

# wait for the run to complete
run = wait_for_run(run, thread.id)

# show information about the run
# should indicate that run status is requires_action
# should contain information about the tools to call
print(run.model_dump_json(indent=2))

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'aasst_NjjVJ5GVWXjJLvYvXtmEQsQR'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

## Helper functions to control lamps

The search_blog function is a helper function that uses the requests library to search the blog. It returns multiple results via a similarity searh in Azure AI Search.

We could query Azure AI Search directly here but the API that is used was already created and running as an Azure Container App.

In [16]:
make_error = False


def set_lamp(lamp="", state=True):
    if make_error:
        return "An error occurred"
    return f"The {lamp} is {'on' if state else 'off'}"

def set_lamp_brightness(lamp="", brightness=100):
    if make_error:
        return "An error occurred"
    return f"The brightness of the {lamp} is set to {brightness}"

## Checking if we need to use a tool

Below we check if we need to use a tool. We assume we need to here. We are not taking into account a scenario where we do not need to use a tool. In reality, we would need to allow for that scenario.

If the assistant indicates we need to use a tool, it will tell use the function name and the arguments to use based on the function definition defined in the assistant. We will then call the search_blog function with the arguments and pass the tool call results back to the assistant.

After passing the tool call results, we run the thread again and show the messages via a json dump.

In [17]:
import json

# we only check for required_action here
# required action means we need to call a tool
if run.required_action:
    # get tool calls and print them
    # check the output to see what tools_calls contains
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    print("Tool calls:", tool_calls)

    # we might need to call multiple tools
    # the assistant API supports parallel tool calls
    # we account for this here although we only have one tool call
    tool_outputs = []
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        # call the function with the arguments provided by the assistant
        if func_name == "set_lamp":
            result = set_lamp(**arguments)
        elif func_name == "set_lamp_brightness":
            result = set_lamp_brightness(**arguments)

        # append the results to the tool_outputs list
        # you need to specify the tool_call_id so the assistant knows which tool call the output belongs to
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": json.dumps(result)
        })

    # now that we have the tool call outputs, pass them to the assistant
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

    print("Tool outputs submitted")

    # now we wait for the run again
    run = wait_for_run(run, thread.id)
else:
    print("No tool calls identified\n")

# show information about the run
print("Run information:")
print("----------------")
print(run.model_dump_json(indent=2), "\n")

# now print all messages in the thread
print("Messages in the thread:")
print("-----------------------")
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(messages.model_dump_json(indent=2))

Tool calls: [RequiredActionFunctionToolCall(id='call_2MhF7oRsIIh3CpLjM7RAuIBA', function=Function(arguments='{"lamp": "living room", "state": true}', name='set_lamp'), type='function'), RequiredActionFunctionToolCall(id='call_SWvFSPllcmVv1ozwRz7mDAD6', function=Function(arguments='{"lamp": "kitchen", "state": true}', name='set_lamp'), type='function'), RequiredActionFunctionToolCall(id='call_auXoTWYehhVQE2YMGOj3bUt0', function=Function(arguments='{"lamp": "living room", "brightness": 50}', name='set_lamp_brightness'), type='function'), RequiredActionFunctionToolCall(id='call_e6ij76rvt70bIhIlwIPVQA2s', function=Function(arguments='{"lamp": "kitchen", "brightness": 50}', name='set_lamp_brightness'), type='function')]
Tool outputs submitted
Run information:
----------------
{
  "id": "run_ciSrRepgIrly3qoDWEY4LLR0",
  "assistant_id": "asst_Z2YGBjhORYJGyPv6AQ4HugzP",
  "cancelled_at": null,
  "completed_at": 1707478823,
  "created_at": 1707478745,
  "expires_at": null,
  "failed_at": null,


In [21]:
import json

messages_json = json.loads(messages.model_dump_json())

def role_icon(role):
    if role == "user":
        return "👤"
    elif role == "assistant":
        return "🤖"

for item in reversed(messages_json['data']):
    # Check the content array
    for content in reversed(item['content']):
        # If there is text in the content array, print it
        if 'text' in content:
            print(role_icon(item["role"]),content['text']['value'], "\n")
        # If there is an image_file in the content, print the file_id
        if 'image_file' in content:
            print("Image ID:" , content['image_file']['file_id'], "\n")

👤 Turn living room lamp and kitchen lamp on. Set both lamps to half brightness. 

🤖 The actions have failed. If you would like to try again or perform another action, please let me know. 

